## Mortgage loans in the United States: Feature Engineering

This project explores trends in mortgage loan applications in the United States during 2018 and 2019. The main objective is to determine which are the factors that are strongest in determining whether a mortgage loan application is accepted or rejected. Data is taken from the United States government's Consumer Financial Protection Bureau and the Federal Financial Institutions Examination Council (FFIEC), which provide access to mortgage loan data, following the 1975 Home Mortgage Disclosure Act (HMDA).

This notebook focuses on the following feature engineering steps in preparation for modelling:

1. Creation of 'dummy' features for categorical variables 
2. Splitting data into training and testing sets
3. Normalization of numerical features

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.float_format = '{:.2f}'.format #Setting changed for better visibility purposes
sns.set()
from scipy.stats import f_oneway
from scipy import stats

In [108]:
filename = '/Users/agm/Desktop/Capstone2/Data/2018_9_reduced.csv'
df = pd.read_csv(filename)

### Creation of 'dummy' variables

A review of our features is due -- we will determine which features to keep and which ones to convert into dummy categories.

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8393516 entries, 0 to 8393515
Data columns (total 24 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   activity_year                      int64  
 1   lei                                object 
 2   state_code                         object 
 3   county_code                        float64
 4   census_tract                       int64  
 5   conforming_loan_limit              object 
 6   action_taken                       int64  
 7   loan_type                          int64  
 8   loan_amount                        float64
 9   loan_to_value_ratio                float64
 10  loan_term                          int64  
 11  property_value                     int64  
 12  occupancy_type                     int64  
 13  total_units                        int64  
 14  income                             float64
 15  debt_to_income_ratio               object 
 16  applicant_ethnicit

**Columns to drop**

0 - activity_year: Ultimately, the year in which the mortgage loan application was submitted should not be a factor to consider.

1 - lei: Who the lender is should also be a matter of indifference;  examining the rates associated with all 3386 lenders is outside the scope of this project. This column will also be dropped.

3 - county_code: While of general interest, the granularity provided by this feature (3217 county codes) is outside the scope of the project. This column will be dropped.

4 - census_tract: As with county_code, the level of granularity provided (72,468) unique census_tract IDs is beyond the scope of this project. 

**Columns to turn into dummies**

2 - state_code: This is a key feature which will be turned into a dummy feature.

5 - conforming_loan_limit: Currently a binary category encoded with strings.

7 - loan_type: Currently a category encoded with four distinct integers.

12 - occupancy_type: Currently a category encoded with three distinct integers. 

15 - debt_to_income_ratio: Binned category with 19 intervals.

16 - applicant_ethnicity-1: Category encoded with 7 distinct integers.

17 - applicant_race-1: Category encoded with 17 distinct integers.

18 - applicant_sex: Category encoded with 3 distinct integers.

19 - applicant_age: Category encoded with 7 age-range bins.

20 - submission_of_application: Category encoded with 4 distinct integers.

**Columns to normalize**

8 - loan_amount: Scalar.

9 - loan_to_value_ratio: Scalar.

10 - loan_term: Scalar.

11 - property_value: Scalar.

13 - total_units: Currently an int with four possible values: 1–4.

14 - income: Scalar.

21 - tract_minority_population_percent: Scalar.

22 - ffiec_msa_md_median_family_income: Scalar.

23 - tract_to_msa_income_percentage: Scalar.

**Other**

6 - action_taken: This is the target variable.

In [109]:
#Dropping columns
df.drop(columns=['activity_year','lei','county_code','census_tract'],inplace=True)

In [110]:
#Converting features currently encoded with integers to categories
df = df.astype({'loan_type': 'category', 'occupancy_type': 'category', 'applicant_ethnicity-1': 'category', 
 'applicant_race-1': 'category', 'applicant_sex': 'category', 'submission_of_application': 'category'})

In [111]:
#Getting dummies
dummies = pd.get_dummies(df[['state_code','conforming_loan_limit','loan_type','occupancy_type',
                             'debt_to_income_ratio','applicant_ethnicity-1','applicant_race-1',
                             'applicant_sex','applicant_age','submission_of_application']], drop_first=True)

#Concatenate dummies to original df
df = pd.concat([df, dummies], axis=1)

#Drop respective columns
df.drop(columns=['state_code','conforming_loan_limit','loan_type','occupancy_type',
                             'debt_to_income_ratio','applicant_ethnicity-1','applicant_race-1',
                             'applicant_sex','applicant_age','submission_of_application'], inplace=True)

In [176]:
#Before normalizing, outliers are handled
def subset_by_quantile(df, column, whisker_width=1.5):
    """Remove outliers from a dataframe by column, including optional 
       whiskers, removing rows for which the column value are 
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    Args:
        df (`:obj:pd.DataFrame`): A pandas dataframe to subset
        column (str): Name of the column to calculate the subset from.
        whisker_width (float): Optional, loosen the IQR filter by a
                               factor of `whisker_width` * IQR.
    Returns:
        (`:obj:pd.DataFrame`): Filtered dataframe
    """
    # Calculate quantiles of interest and IQR
    q1 = df[column].quantile(0.01)                 
    q3 = df[column].quantile(0.99)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter]   

In [177]:
#Drop rows where loan_term > 970
df = df[df['loan_term'] <= 970]

#Apply quantile filters
for column in ['loan_amount', 'loan_to_value_ratio','property_value','income']:
    df = subset_by_quantile(df, column, whisker_width=1.5)

In [183]:
#Before normalizing, we split our data set
from sklearn.model_selection import train_test_split

X = df.drop(columns=['action_taken'])
y = df[['action_taken']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=58)

In [195]:
#Now we normalize
from sklearn.preprocessing import MinMaxScaler

#Applying scaler on train set
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

#Applying scaler on test set
X_test = scaler.transform(X_test)